In [3]:
# data analysis libraries
import pandas as pd
import numpy as np

# system libraries
import glob
from pathlib import Path
import random
import time

# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning libraries

# additional libraries
from tqdm import tqdm
# import functools




Data prep to get the input files in a single tabular form that is ready to split into training, validation and test datasets before diving into EDA for the training set

In [ ]:
# # from decorators import timer
# from package import decorators

def read_random_csv_files(path,ftype='csv',n=1) -> pd.DataFrame:
    """
    Read n random files from a directory and return a DataFrame.
    
    Parameters:
    path (str): The directory regex containing the files.
    ftype (str): The file type to read. Default is 'csv'.
    n (int): The number of random CSV files to read. Default is 1.
    
    Returns:
    pd.DataFrame: A DataFrame containing the data from the random CSV files.
    """
    # set separator
    if ftype == 'csv':
        sep = ','
    elif ftype == 'tsv':
        sep = '\t'
    else:
        raise ValueError("Unsupported file type. Use 'csv' or 'tsv'.")
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(f"{path}")
    
    # Randomly select n CSV files
    random.seed(42)
    selected_files = random.sample(csv_files, n)
    print(f"Selected files: {selected_files}")
    
    # Read and concatenate the selected CSV files into a single DataFrame
    df = pd.concat([pd.read_csv(file, sep=sep) for file in selected_files], ignore_index=True)
    
    return df


def get_df_head_tail_shape(df: pd.DataFrame) -> None:
    """
    Print the information of a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to print its head, tail, shape.
    
    Returns:
    None
    """
    
    print("\nDataFrame Shape:")
    print(df.shape)
    print("\nDataFrame Head:")
    print(df.head())
    print("\nDataFrame Tail:")
    print(df.tail())


In [5]:
# how many patients ? Count number of records 
file_count = 0
for filename in glob.glob("../set-a/*.txt"):
    file_count += 1

print(f"Total {file_count} records i.e. patients in the datasetA")

Total 4000 records i.e. patients in the datasetA


In [6]:
# Randomly select a file from the directory
df = read_random_csv_files("../set-a/*.txt")

# Print the DataFrame information
get_df_head_tail_shape(df)

Selected files: ['../set-a/132933.txt']

DataFrame Shape:
(347, 3)

DataFrame Head:
    Time Parameter     Value
0  00:00  RecordID  132933.0
1  00:00       Age      54.0
2  00:00    Gender       0.0
3  00:00    Height     170.2
4  00:00   ICUType       4.0

DataFrame Tail:
      Time Parameter  Value
342  45:29    SysABP  116.0
343  45:29      Temp   38.1
344  45:29     Urine   80.0
345  47:21      FiO2    0.5
346  47:21  MechVent    1.0


- The dataset instructions mentioned that the EMR have `Time` recorded up to 48hrs. In the file tail view, I see that the timestamp is in HH:MM format. Pandas doesn't play nice with HH over 24hrs. Converting this to append `Day` during data prep (long-to-wide)  

In [7]:
# FIXME: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

setA_concat_all = pd.DataFrame()
record_stats = pd.DataFrame(columns=['RecordID', 'Parameter', 'min', 'max', 'mean','median', 'std'])

for record_name in tqdm(glob.glob("../set-a/*.txt"), desc="Reading file"): 
    record = pd.read_csv(record_name)
    # Ensure Time is in the expected hh:mm:ss format
    record['Time_dhm'] = record['Time'].apply(lambda x: f"{x}:00" if len(x.split(':')) == 2 else x)
    record['Time_dhm']  = pd.to_timedelta(record['Time_dhm'])
    record.drop(['Time'], axis=1, inplace=True)
    # concatanate to setA_concat_all
    setA_concat_all = pd.concat([setA_concat_all,record]) 

    
    recordID = record[record['Parameter'] =='RecordID']['Value'][0]
    for measurement in record['Parameter'].unique():
        if measurement != 'RecordID':
        # print(measurement)
        # print(record[record['Parameter'] == measurement])
            record_stats = pd.concat([record_stats, pd.DataFrame([[recordID, measurement,
                                                                record[record['Parameter'] == measurement]['Value'].min(),
                                                                record[record['Parameter'] == measurement]['Value'].max(),
                                                                record[record['Parameter'] == measurement]['Value'].mean(),
                                                                record[record['Parameter'] == measurement]['Value'].median(),
                                                                record[record['Parameter'] == measurement]['Value'].std()]], 
                                                                columns=['RecordID', 'Parameter', 'min', 'max', 'mean','median', 'std'])])

    

Reading file:   0%|          | 0/4000 [00:00<?, ?it/s]/tmp/ipykernel_279552/675773938.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  record_stats = pd.concat([record_stats, pd.DataFrame([[recordID, measurement,
Reading file: 100%|██████████| 4000/4000 [09:01<00:00,  7.39it/s]


In [8]:
setA_concat_all.shape
setA_concat_all.head()

record_stats.shape
record_stats.head()
record_stats.tail()

# # write the concatenated DataFrames to CSV files. Commented out to avoid overwriting
# setA_concat_all.to_csv('../out_files/setA_concat_all.csv', index=False)
# record_stats.to_csv('../out_files/record_stats.csv', index=False)

(1757980, 3)

,Parameter,Value,Time_dhm
0,RecordID,137680.0,0 days
1,Age,84.0,0 days
2,Gender,1.0,0 days
3,Height,-1.0,0 days
4,ICUType,4.0,0 days


(122993, 7)

,RecordID,Parameter,min,max,mean,median,std
0,137680.0,Age,84.0,84.0,84.0,84.0,NaN
0,137680.0,Gender,1.0,1.0,1.0,1.0,NaN
0,137680.0,Height,-1.0,-1.0,-1.0,-1.0,NaN
0,137680.0,ICUType,4.0,4.0,4.0,4.0,NaN
0,137680.0,Weight,80.4,80.4,80.4,80.4,NaN


,RecordID,Parameter,min,max,mean,median,std
0,140497.0,Lactate,1.30,1.30,1.300000,1.300,NaN
0,140497.0,FiO2,0.35,1.00,0.653571,0.700,0.152498
0,140497.0,pH,7.38,7.41,7.395000,7.395,0.021213
0,140497.0,PaCO2,30.00,32.00,31.000000,31.000,1.414214
0,140497.0,PaO2,77.00,81.00,79.000000,79.000,2.828427


In [9]:
# confirm that 4k records are present
record_stats['RecordID'].nunique()

4000

In [10]:
# pivot record_stats to wide format
record_stats_pivot = record_stats.pivot(index='RecordID', columns='Parameter', values=['min', 'max', 'mean','median', 'std'])
record_stats_pivot.columns = ['_'.join(col).strip() for col in record_stats_pivot.columns.values]
record_stats_pivot.reset_index(inplace=True)
record_stats_pivot.head()
record_stats_pivot.shape

,RecordID,min_ALP,min_ALT,min_AST,min_Age,min_Albumin,min_BUN,min_Bilirubin,min_Cholesterol,min_Creatinine,...,std_RespRate,std_SaO2,std_SysABP,std_Temp,std_TroponinI,std_TroponinT,std_Urine,std_WBC,std_Weight,std_pH
0,132539.0,NaN,NaN,NaN,54.0,NaN,8.0,NaN,NaN,0.7,...,3.216898,NaN,NaN,0.935385,NaN,NaN,171.870307,1.272792,NaN,NaN
1,132540.0,NaN,NaN,NaN,76.0,NaN,16.0,NaN,NaN,0.8,...,NaN,2.041241,16.338979,0.986234,NaN,NaN,161.509760,3.350124,9.209775e-01,0.037796
2,132541.0,105.0,75.0,164.0,44.0,2.3,3.0,2.8,NaN,0.3,...,NaN,NaN,12.153017,0.642112,NaN,NaN,93.216670,1.322876,2.868367e-14,0.017321
3,132543.0,105.0,12.0,15.0,68.0,4.4,10.0,0.2,NaN,0.7,...,3.410542,NaN,NaN,0.490160,NaN,NaN,224.953699,1.873499,4.299234e-14,NaN
4,132545.0,NaN,NaN,NaN,88.0,3.3,25.0,NaN,NaN,1.0,...,3.263814,NaN,NaN,0.470865,NaN,NaN,39.383198,0.707107,NaN,NaN


(4000, 206)

In [11]:
# # write the concatenated DataFrames to CSV files. Commented out to avoid overwriting
# record_stats_pivot.to_csv('../out_files/record_stats_pivot.csv', index=False)

In [12]:
# Read and combine outcome data
outcome = pd.read_csv("../Outcomes-a.txt")
outcome.head()
outcome.shape
outcome['RecordID'].nunique()

,RecordID,SAPS-I,SOFA,Length_of_stay,Survival,In-hospital_death
0,132539,6,1,5,-1,0
1,132540,16,8,8,-1,0
2,132541,21,11,19,-1,0
3,132543,7,1,9,575,0
4,132545,17,2,4,918,0


(4000, 6)

4000

The instructions mentioned Survival definition & constraint as

    - Survival > Length of stay  ⇒  Survivor
    - Survival = -1  ⇒  Survivor
    - 2 ≤ Survival ≤ Length of stay  ⇒  In-hospital death
Confirming that the Survival column doesn't skip/miss any that should be recorded as 1


In [13]:
# count of rows where Survival is >=2 but less than Length_of_stay i.e. in-hospital death
outcome[(outcome['Survival'] >= 2) & (outcome['Survival'] < outcome['Length_of_stay'])].shape[0]
# get list of recordIDs where Survival is >=2 but less than Length_of_stay (should be same as above) i.e. no recordID that should be flagged for in-hospital death is missing
recordid_subset_criteria = outcome[(outcome['Survival'] >= 2) & (outcome['Survival'] < outcome['Length_of_stay'])]['RecordID'].unique()
len(recordid_subset_criteria)

515

515

In [14]:
# list of recordIDs where In-hospital_death is 1
recordid_subset_death = outcome[outcome['In-hospital_death'] == 1]['RecordID'].unique()
len(recordid_subset_death)
# ~14% of patients died in the hospital. This is sign that the dataset is moderately imbalanced. This will impact the split strategy.


554

In [15]:
# merge the two dataframes on RecordID
merged_df = pd.merge(record_stats_pivot, outcome, on='RecordID', how='left')
merged_df.shape
merged_df.head()

(4000, 211)

,RecordID,min_ALP,min_ALT,min_AST,min_Age,min_Albumin,min_BUN,min_Bilirubin,min_Cholesterol,min_Creatinine,...,std_TroponinT,std_Urine,std_WBC,std_Weight,std_pH,SAPS-I,SOFA,Length_of_stay,Survival,In-hospital_death
0,132539.0,NaN,NaN,NaN,54.0,NaN,8.0,NaN,NaN,0.7,...,NaN,171.870307,1.272792,NaN,NaN,6,1,5,-1,0
1,132540.0,NaN,NaN,NaN,76.0,NaN,16.0,NaN,NaN,0.8,...,NaN,161.509760,3.350124,9.209775e-01,0.037796,16,8,8,-1,0
2,132541.0,105.0,75.0,164.0,44.0,2.3,3.0,2.8,NaN,0.3,...,NaN,93.216670,1.322876,2.868367e-14,0.017321,21,11,19,-1,0
3,132543.0,105.0,12.0,15.0,68.0,4.4,10.0,0.2,NaN,0.7,...,NaN,224.953699,1.873499,4.299234e-14,NaN,7,1,9,575,0
4,132545.0,NaN,NaN,NaN,88.0,3.3,25.0,NaN,NaN,1.0,...,NaN,39.383198,0.707107,NaN,NaN,17,2,4,918,0


In [16]:
# # write the concatenated DataFrames to CSV files. Commented out to avoid overwriting
# merged_df.to_csv('../out_files/record_stats_outcome_merged.csv', index=False)